In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-156272
Azure region: southcentralus
Subscription id: 2c48c51c-bd47-40d4-abbe-fb8eabd19c8c
Resource group: aml-quickstarts-156272


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cluster_name = "cluster-project"
try:
    target = ComputeTarget(workspace=ws, name=cluster_name)
    print("Same cluster name compute exists. Reusing the same")
except: 
    print("Compute with same cluster name doesn't exist and hence creating NEW")
    config_compute = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes = 10)
    target = ComputeTarget.create(ws, cluster_name, config_compute)

target.wait_for_completion(show_output=True)
print(target.get_status().serialize())


Compute with same cluster name doesn't exist and hence creating NEW
InProgress......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Resizing', 'allocationStateTransitionTime': '2021-08-30T08:50:12.373000+00:00', 'errors': None, 'creationTime': '2021-08-30T08:50:11.976313+00:00', 'modifiedTime': '2021-08-30T08:50:37.658299+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 10, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [5]:
print(2+3)

5


In [6]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    '--C' : choice(0.001,0.01,0.1,1,10,50,100,150,200,300,400, 500,1000),
    '--max_iter' : choice(50,100,150,200,300)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(
    source_directory='./',
    compute_target=target,
    vm_size='Standard_D2_V2',
    entry_script='train.py'
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    hyperparameter_sampling=ps,
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    primary_metric_name='Accuracy',
    policy=ps,
    estimator=est,
    max_total_runs=20,
    max_concurrent_runs=4

)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [8]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
run_hyp = exp.submit(hyperdrive_config)
RunDetails(run_hyp).show()

run_hyp.wait_for_completion(show_output = True)
assert(run_hyp.get_status() =="Completed")

ClientRequestError: Error occurred in request., RetryError: HTTPSConnectionPool(host='southcentralus.experiments.azureml.net', port=443): Max retries exceeded with url: /hyperdrive/v1.0/subscriptions%2F2c48c51c-bd47-40d4-abbe-fb8eabd19c8c%2FresourceGroups%2Faml-quickstarts-156272%2Fproviders%2FMicrosoft.MachineLearningServices%2Fworkspaces%2Fquick-starts-ws-156272%2Fexperiments%2Fudacity-project/runs (Caused by ResponseError('too many 500 error responses',))

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###